In [1]:
%reset -f

In [2]:
%reload_ext sql
%sql sqlite:///../ontology/tmp/merged.db

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///../ontology/tmp/merged.db
Done.


name
term_association
has_oio_synonym_statement
anonymous_expression
anonymous_class_expression
anonymous_property_expression
anonymous_individual_expression
owl_restriction
owl_complex_axiom
prefix
rdf_list_statement


In [4]:
%sql SELECT * FROM prefix WHERE prefix LIKE "OM%" OR prefix LIKE "OR%" ORDER BY prefix;

 * sqlite:///../ontology/tmp/merged.db
Done.


prefix,base
OMIM,https://omim.org/entry/
OMIMPS,https://omim.org/phenotypicSeries/PS
OMIT,http://purl.obolibrary.org/obo/OMIT_
OMO,http://purl.obolibrary.org/obo/OMO_
OMP,http://purl.obolibrary.org/obo/OMP_
OMRSE,http://purl.obolibrary.org/obo/OMRSE_
ORNASEQ,http://purl.obolibrary.org/obo/ORNASEQ_
Orphanet,http://www.orpha.net/ORDO/Orphanet_


In [5]:
import pandas as pd
from os.path import join
import textdistance

def save_csv(query, fn):
    dir_name = "dataframes"
    df = pd.DataFrame(query)
    df.to_csv(join(dir_name,fn), sep='\t', index = False)
    return df

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_value.lower(), x.object_value.lower()\
                 ),\
                axis=1,\
            )\
           )

In [ ]:
query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" AND object LIKE "%ICD10%"
mondo_2_icd_df = save_csv(query, "mondo_2_icd.tsv")
mondo_2_icd_df.head()


In [8]:
# query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" OR subject LIKE "%ICD10%" AND object LIKE "%MONDO:%" OR object LIKE "%ICD10%"
# mondo_icd_2_mondo_icd_df = save_csv(query, "mondo_icd_2_mondo_icd.tsv")
# mondo_icd_2_mondo_icd_df.head()
%sql SELECT * FROM statements \
WHERE subject LIKE "%MONDO:%" OR subject LIKE "%ICD10%" \
AND object LIKE "%MONDO:%" OR object LIKE "%ICD10%" LIMIT 10


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
MONDO:exclusionReason,MONDO:exclusionReason,rdf:type,owl:AnnotationProperty,None,None,None,None
MONDO:0100332,MONDO:0100332,rdfs:label,None,disease has primary infectious agent,None,None,None
MONDO:0100332,MONDO:0100332,oio:id,None,MONDO:0100332,None,None,None
MONDO:0100332,MONDO:0100332,oio:created_by,None,https://orcid.org/0000-0001-5208-3432,None,None,None
MONDO:0100332,MONDO:0100332,rdfs:subPropertyOf,RO:0014001,None,None,None,None
MONDO:0100332,MONDO:0100332,rdf:type,owl:ObjectProperty,None,None,None,None
MONDO:0100333,MONDO:0100333,rdfs:label,None,disease caused by reactivation of latent infectious agent,None,None,None
MONDO:0100333,MONDO:0100333,oio:id,None,MONDO:0100333,None,None,None
MONDO:0100333,MONDO:0100333,oio:created_by,None,https://orcid.org/0000-0001-5208-3432,None,None,None
MONDO:0100333,MONDO:0100333,rdfs:subPropertyOf,RO:0014001,None,None,None,None


In [9]:
%sql SELECT * FROM statements \
WHERE subject IN ("MONDO:0007111", "DOID:0080964", "OMIM:105800")\
AND object IN ("MONDO:0007111","OMIM:105800", "DOID:0080964")

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
DOID:0080964,DOID:0080964,skos:exactMatch,OMIM:105800,None,None,None,None
MONDO:0007111,MONDO:0007111,skos:exactMatch,OMIM:105800,None,None,None,None


In [10]:
# query = %sql SELECT * FROM statements WHERE (subject LIKE "%MONDO:%" OR subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 10
# mondo_icd_label_exact_synonym_df = save_csv(query, "mondo_icd_label_exact_synonym.tsv")
# mondo_icd_label_exact_synonym_df.head()
%sql SELECT * FROM statements WHERE (subject LIKE "%MONDO:%" OR subject LIKE "%icd10cm%")\
    AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 5


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
ICD10CM:A00-A09,ICD10CM:A00-A09,oio:hasExactSynonym,None,intestinal infectious diseases,None,None,None
ICD10CM:A00-B99,ICD10CM:A00-B99,oio:hasExactSynonym,None,diseases generally recognized as communicable or transmissible,None,en,None
ICD10CM:A00-B99,ICD10CM:A00-B99,oio:hasExactSynonym,None,certain infectious and parasitic diseases,None,None,None
ICD10CM:A00.0,ICD10CM:A00.0,oio:hasExactSynonym,None,Classical cholera,None,en,None
ICD10CM:A00.1,ICD10CM:A00.1,oio:hasExactSynonym,None,Cholera eltor,None,en,None


In [11]:
%sql SELECT * FROM statements WHERE (subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 10


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
ICD10CM:A00-A09,ICD10CM:A00-A09,oio:hasExactSynonym,None,intestinal infectious diseases,None,None,None
ICD10CM:A00-B99,ICD10CM:A00-B99,oio:hasExactSynonym,None,diseases generally recognized as communicable or transmissible,None,en,None
ICD10CM:A00-B99,ICD10CM:A00-B99,oio:hasExactSynonym,None,certain infectious and parasitic diseases,None,None,None
ICD10CM:A00.0,ICD10CM:A00.0,oio:hasExactSynonym,None,Classical cholera,None,en,None
ICD10CM:A00.1,ICD10CM:A00.1,oio:hasExactSynonym,None,Cholera eltor,None,en,None
ICD10CM:A01.0,ICD10CM:A01.0,oio:hasExactSynonym,None,Infection due to Salmonella typhi,None,en,None
ICD10CM:A01.02,ICD10CM:A01.02,oio:hasExactSynonym,None,Typhoid myocarditis,None,en,None
ICD10CM:A01.02,ICD10CM:A01.02,oio:hasExactSynonym,None,Typhoid endocarditis,None,en,None
ICD10CM:A01.4,ICD10CM:A01.4,oio:hasExactSynonym,None,Infection due to Salmonella paratyphi NOS,None,en,None
ICD10CM:A02,ICD10CM:A02,oio:hasExactSynonym,None,infection or foodborne intoxication due to any Salmonella species other than S. typhi and S. paratyphi,None,en,None


In [12]:
%sql SELECT * FROM statements WHERE (subject == "ICD10CM:D35.2") LIMIT 10


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
ICD10CM:D35.2,ICD10CM:D35.2,rdf:type,owl:Class,None,None,None,None
ICD10CM:D35.2,ICD10CM:D35.2,rdfs:label,None,Benign neoplasm of pituitary gland,None,en,None
ICD10CM:D35.2,ICD10CM:D35.2,rdfs:subClassOf,ICD10CM:D35,None,None,None,None


In [13]:
%sql SELECT * FROM statements WHERE (object == "ICD10CM:D35.2") LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
MONDO:0021439,MONDO:0021439,skos:exactMatch,ICD10CM:D35.2,None,None,None,None
MONDO:0021439,_:riog00506967,owl:annotatedTarget,ICD10CM:D35.2,None,None,None,None


In [14]:
%sql SELECT * FROM statements WHERE (subject == "MONDO:0017359")

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
MONDO:0017359,MONDO:0017359,IAO:0000115,None,"A group of five inherited disorders caused by mutations in the AUH, DNAJC19, OPA3, and TAZ genes. The disorders are characterized by impairment in the function of mitochondria, resulting in the accumulation and excretion of 3-methylglutaconic acid, and the presence of 3-methylglutaric acid in the urine.",None,None,None
MONDO:0017359,MONDO:0017359,obo:mondo#should_conform_to,obo:mondo/patterns/OMIM_phenotypic_series.yaml,None,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,DOID:0060336,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,ICD10CM:E71.111,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,MESH:C579867,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,NCIT:C98678,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,OMIMPS:250950,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,Orphanet:289902,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,SCTID:237950009,None,None,None
MONDO:0017359,MONDO:0017359,oio:hasDbXref,None,UMLS:C3696376,None,None,None


In [15]:
%sql SELECT * FROM entailed_edge WHERE object LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
ICD10WHO:N23,rdfs:subClassOf,ICD10WHO:N23
ICD10CM:T42.5X6A,rdfs:subClassOf,ICD10CM:T42.5X6A
ICD10CM:S06.4X6D,rdfs:subClassOf,ICD10CM:S06.4X6D
ICD10CM:N44.03,rdfs:subClassOf,ICD10CM:N44.03
ICD10CM:S82.042S,rdfs:subClassOf,ICD10CM:S82.042S
ICD10CM:T83.121A,rdfs:subClassOf,ICD10CM:T83.121A
ICD10CM:V87.6XXD,rdfs:subClassOf,ICD10CM:V87.6XXD
ICD10WHO:Y41.0,rdfs:subClassOf,ICD10WHO:Y41.0
ICD10CM:S14.132,rdfs:subClassOf,ICD10CM:S14.13
ICD10CM:S14.138S,rdfs:subClassOf,ICD10CM:S14.13


In [16]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%MONDO%" AND object LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [17]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" AND object LIKE "%MONDO%"  LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [18]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
ICD10WHO:N23,rdfs:subClassOf,ICD10WHO:N23
ICD10CM:T42.5X6A,rdfs:subClassOf,ICD10CM:T42.5X6A
ICD10CM:S06.4X6D,rdfs:subClassOf,ICD10CM:S06.4X6D
ICD10CM:N44.03,rdfs:subClassOf,ICD10CM:N44.03
ICD10CM:S82.042S,rdfs:subClassOf,ICD10CM:S82.042S
ICD10CM:T83.121A,rdfs:subClassOf,ICD10CM:T83.121A
ICD10CM:V87.6XXD,rdfs:subClassOf,ICD10CM:V87.6XXD
ICD10WHO:Y41.0,rdfs:subClassOf,ICD10WHO:Y41.0
ICD10CM:S14.132,rdfs:subClassOf,ICD10CM:S14.13
ICD10CM:S14.138S,rdfs:subClassOf,ICD10CM:S14.13


In [19]:
%sql SELECT * FROM rdfs_label_statement LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language,graph
IAO:0000111,IAO:0000111,rdfs:label,None,editor preferred term,None,en,None
IAO:0000112,IAO:0000112,rdfs:label,None,example of usage,None,en,None
IAO:0000113,IAO:0000113,rdfs:label,None,in branch,None,en,None
IAO:0000114,IAO:0000114,rdfs:label,None,has curation status,None,en,None
IAO:0000115,IAO:0000115,rdfs:label,None,definition,None,en,None
IAO:0000115,IAO:0000115,rdfs:label,None,definition,None,None,None
IAO:0000116,IAO:0000116,rdfs:label,None,editor note,None,en,None
IAO:0000117,IAO:0000117,rdfs:label,None,term editor,None,en,None
IAO:0000118,IAO:0000118,rdfs:label,None,alternative term,None,en,None
IAO:0000118,IAO:0000118,rdfs:label,None,alternative label,None,en,None


## Post-process some DataFrames

In [20]:
full_table_query = %sql SELECT subject, predicate, object, value, datatype FROM rdfs_label_statement
full_table_df = pd.DataFrame(full_table_query)
full_table_df.head()

 * sqlite:///../ontology/tmp/merged.db
Done.


,subject,predicate,object,value,datatype
0,IAO:0000111,rdfs:label,None,editor preferred term,None
1,IAO:0000112,rdfs:label,None,example of usage,None
2,IAO:0000113,rdfs:label,None,in branch,None
3,IAO:0000114,rdfs:label,None,has curation status,None
4,IAO:0000115,rdfs:label,None,definition,None


In [21]:
full_table_df_with_values = full_table_df[full_table_df['value'].values != None]
subject_value_df = full_table_df_with_values[['subject', "value"]]
subject_value_df.head()

,subject,value
0,IAO:0000111,editor preferred term
1,IAO:0000112,example of usage
2,IAO:0000113,in branch
3,IAO:0000114,has curation status
4,IAO:0000115,definition


In [ ]:
cols_to_drop = ['stanza','value', 'datatype','language']
mondo_2_icd_df_subset = mondo_2_icd_df.drop(columns=cols_to_drop)
mondo_2_icd_df_subset.head()

In [ ]:
mondo_2_icd_subset_with_values = mondo_2_icd_df_subset\
                                .merge(subject_value_df, how="left", on="subject")\
                                .rename(columns={'value': "subject_value"})
mondo_2_icd_subset_with_values.head()


In [ ]:
mondo_2_icd_values_df = mondo_2_icd_subset_with_values\
                        .merge(subject_value_df, how="left", left_on="object", right_on="subject")\
                        .rename(columns={'value': "object_value", "subject_x": "subject"})\
                        .drop(columns=['subject_y'])\
                        .fillna("")
        

mondo_2_icd_values_df.head()


In [ ]:
mondo_2_icd_values_df = mondo_2_icd_values_df[['subject', 'subject_value', 'predicate', 'object', 'object_value' ]]
mondo_2_icd_values_df.to_csv(join("dataframes","mondo2icd_with_values.tsv"), sep='\t', index=False)


## Subject and Object value text-distances

In [ ]:
# Add Levenshtein distance [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "levenshtein_dist", textdistance.levenshtein.distance)

In [ ]:
# Add Jaccard Index [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "jaccard_index", textdistance.jaccard.distance)

In [ ]:
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "monge_elkan", textdistance.monge_elkan.distance)
mondo_2_icd_values_df.to_csv(join("dataframes","mondo2icd_with_distances.tsv"), sep='\t', index=False)
mondo_2_icd_values_df.head()

In [ ]:
condition_1 = mondo_2_icd_values_df['levenshtein_dist'] < 10
condition_2 = mondo_2_icd_values_df['predicate'] != 'skos:exactMatch'
condition_3 = mondo_2_icd_values_df['object_value'] != ""

mondo_2_icd_values_df.loc[(condition_1) & (condition_2) & (condition_3)]